In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cp -r "drive/My Drive/KS" .

In [ ]:
!python -m pip install -r "./KS/requirements.txt"

In [8]:
import os
import argparse
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from KS.src.dataset import *
from KS.src.model import EfficientDet
import shutil
import numpy as np
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR,ReduceLROnPlateau
# custom module
from KS.util import *
import datetime
from torchsummary import summary
torch.backends.cudnn.benchmark=True

device = torch.device("cuda")

In [60]:
class TestDataset(Dataset):
    def __init__(self,num):
      self.x = ['./KS/data/test_f/' + str(i + 1) + '.png' for i in range(num)]

    def __len__(self):
      return len(self.x)

    def __getitem__(self, item):
      self.x[item] = cv2.imread(self.x[item])       
      self.x[item] = np.transpose(self.x[item], (2, 0, 1)) / 255.
      self.x[item] = torch.tensor(self.x[item])
      self.x[item] = Normalizer(self.x[item])
      return self.x[item]
    

In [ ]:

torch.cuda.manual_seed(1)
#load dataset
datasets = TestDataset(1)
data_loader = DataLoader(datasets, batch_size = 1, shuffle = False,drop_last = False, num_workers = 1)
#load model
model = EfficientDet(num_classes=10)
model = model.to(device)
model = nn.DataParallel(model)

#resume
checkpoint = torch.load("./KS/checkpoint/14_resnet34/model_32_0.6379526853561401.pth", map_location='cpu')
model.load_state_dict(checkpoint['model'])

model.eval()

In [55]:
print(device)

a = torch.cuda.is_available()
print(a)

cuda
True


In [70]:
%%timeit
with torch.no_grad():
  for images in data_loader: 
      image = images.to(device).float()
      [nms_scores, nms_class, transformed_anchors]= model([image])

10 loops, best of 3: 120 ms per loop


In [66]:
!nvidia-smi

Thu Nov 26 14:51:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    31W /  70W |   3363MiB / 15079MiB |      6%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------